# Appendix: Import needed modules 

In [1]:
import re, collections
from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords
from spacy.en import English ##Note you'll need to install Spacy and download its dependencies
parser = English()
import string

In [2]:
# A custom stoplist
STOPLIST = set(stopwords.words('english') + ["n't", "'s", "'m", "ca"] + list(ENGLISH_STOP_WORDS))
# List of symbols we don't care about
SYMBOLS = " ".join(string.punctuation).split(" ") + ["-----", "---", "...", "“", "”", "'ve"]

In [3]:
import re, collections

def words(text): return re.findall('[a-z]+', text.lower()) 

def train(features):
    model = collections.defaultdict(lambda: 1)
    for f in features:
        model[f] += 1
    return model

NWORDS = train(words(open('C:/Users/kenndanielso/Documents/Github/mcnulty_yelp/data/bigtext.txt').read()))

alphabet = 'abcdefghijklmnopqrstuvwxyz'

def edits1(word):
   splits     = [(word[:i], word[i:]) for i in range(len(word) + 1)]
   deletes    = [a + b[1:] for a, b in splits if b]
   transposes = [a + b[1] + b[0] + b[2:] for a, b in splits if len(b)>1]
   replaces   = [a + c + b[1:] for a, b in splits for c in alphabet if b]
   inserts    = [a + c + b     for a, b in splits for c in alphabet]
   return set(deletes + transposes + replaces + inserts)

def known_edits2(word):
    return set(e2 for e1 in edits1(word) for e2 in edits1(e1) if e2 in NWORDS)

def known(words): return set(w for w in words if w in NWORDS)

def correct(word):
    candidates = known([word]) or known(edits1(word)) or known_edits2(word) or [word]
    return max(candidates, key=NWORDS.get)

In [20]:
# A custom function to clean the text before sending it into the vectorizer
def cleanText(text):
    # get rid of newlines
    text = text.strip().replace("\n", " ").replace("\r", " ")
    
    # replace twitter @mentions
#     mentionFinder = re.compile(r"@[a-z0-9_]{1,15}", re.IGNORECASE)
#     text = mentionFinder.sub("@MENTION", text)
    text = re.sub('[^a-zA-Z0-9 ]','',text)
    # replace HTML symbols
    text = text.replace("&amp;", "and").replace("&gt;", ">").replace("&lt;", "<")
    
    # lowercase
    text = text.lower()
#     text = correct(text)

    return text

# A custom function to tokenize the text using spaCy
# and convert to lemmas
def tokenizeText(sample):

    # get the tokens using spaCy
    tokens = parser(str(TextBlob(sample).correct()))

    # lemmatize
    lemmas = []
    for tok in tokens:
        lemmas.append(tok.lemma_.lower().strip() if tok.lemma_ != "-PRON-" else tok.lower_)
    tokens = lemmas

    # stoplist the tokens
    tokens = [tok for tok in tokens if tok not in STOPLIST]

    # stoplist symbols
    tokens = [tok for tok in tokens if tok not in SYMBOLS]

    # remove large strings of whitespace
    while "" in tokens:
        tokens.remove("")
    while " " in tokens:
        tokens.remove(" ")
    while "\n" in tokens:
        tokens.remove("\n")
    while "\n\n" in tokens:
        tokens.remove("\n\n")

    return tokens

In [5]:
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from nltk.corpus import stopwords

stops = set(stopwords.words('english'))

import pandas as pd
df = pd.read_csv('C:/Users/kenndanielso/Documents/Github/mcnulty_yelp/data/sentence_raw.csv',encoding = "ISO-8859-1")

# Download data and munge munge

In [21]:
df2 = df[['sentence','category']]

In [22]:
##Collapse some
df2[df2['category']=="wait"] = "service" 
df2[df2['category']=="value"] = "overall"


C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Anaconda3\lib\site-packages\pandas\core\frame.py:2363: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.ix._setitem_with_indexer(indexer, value)
C:\Anaconda3\lib\site-packages\pandas\core\frame.py:2343: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_array(key, value)
C:\Ana

In [23]:
df2.category.value_counts()/df2.shape[0]

food        0.414732
overall     0.263298
others      0.138862
service     0.123872
ambiance    0.059236
Name: category, dtype: float64

In [24]:
df2.sentence = df2.sentence.apply(cleanText)

C:\Anaconda3\lib\site-packages\pandas\core\generic.py:2698: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


# Feature Generation

In [26]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = CountVectorizer(ngram_range=(1,3),min_df=3, max_features=1000,tokenizer=tokenizeText)
tfvectorizer = TfidfVectorizer(ngram_range=(1,3),min_df = 3,max_features=1000,tokenizer=tokenizeText,sublinear_tf=True)

In [ ]:
## Gets the count of each word in each sentence (Count Vectorizer)
from sklearn.decomposition import TruncatedSVD
import pickle
countfeature = vectorizer.fit_transform(df2.sentence)
# lsa_count = TruncatedSVD(n_components=50,n_iter=100).fit_transform(countfeature)
# lsa_count_df = pd.DataFrame(lsa_count)
featuredf_count = pd.DataFrame(countfeature.A, columns=vectorizer.get_feature_names())
df3_count = pd.concat((df2,featuredf_count),axis=1)
# df3_lsa_count = pd.concat((df2,lsa_count_df),axis=1)
print(df3_count.info())
# print(df3_lsa_count.info())
df3_count.to_pickle("df3_count.pkl")
# df3_lsa_count.to_pickle("df3_lsa_count.pkl")

In [ ]:
##TFIDF Vectorizer (slow)
tffeature = tfvectorizer.fit_transform(df2.sentence)
featuredf_tf = pd.DataFrame(tffeature.A, columns=tfvectorizer.get_feature_names())
# lsa_tf = TruncatedSVD(n_components=50,n_iter=100).fit_transform(tffeature)
# lsa_tf_df = pd.DataFrame(lsa_tf)
df3_tf = pd.concat((df2,featuredf_tf),axis=1)
# df3_lsa_tf = pd.concat((df2,lsa_tf_df),axis=1)
print(df3_tf.info())
# print(df3_lsa_tf.info())
df3_tf.to_pickle("df3_tf.pkl")
# df3_lsa_tf.to_pickle("df3_lsa_tf.pkl")

# Building the model and in-training-set accuracy

In [ ]:
from sklearn.cross_validation import train_test_split 

In [59]:
##Split into train and test at 75/25
train, test = train_test_split(df3_count,test_size = 0.25, random_state=1)

TypeError: Expected sequence or array-like, got estimator                                                 sentence  category  able  \
0                                      thanks  nice spot  ambiance     0   
1      this is a new pho place and the only phos rest...  ambiance     0   
2               ambiance was bright chic and comfortable  ambiance     0   
3           i remember liking this place in my past life  ambiance     0   
4                              the lighting is very dark  ambiance     0   
5                                very chill and inviting  ambiance     0   
6      and the place is quiet during lunchtime during...  ambiance     0   
7      there is a bar and grill a arcade a  story laz...  ambiance     0   
8                                  casual fun atmosphere  ambiance     0   
9      its a small restaurant with additional outdoor...  ambiance     0   
10     its a good atmosphere for sure but the sushi c...  ambiance     0   
11     this place was amazzzzi

In [ ]:
msk = np.random.random(df3_count.shape[0]) < 0.8

In [ ]:
train = df3_count[msk]
test = df3_count[~msk]

In [ ]:
##Split X & Y
X_train = train.iloc[:,2:]
Y_train = train.iloc[:,1]
X_test = test.iloc[:,2:]
Y_test = test.iloc[:,1]

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

## NaiveBayes
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB().fit(X_train,Y_train)
nb_Y_pred = nb.predict(X_test)
print("NB Accuracy: ",np.mean(nb_Y_pred == np.array(Y_test)))
print(classification_report(Y_test,nb_Y_pred))

## Logistic
from sklearn.linear_model import LogisticRegression
log = LogisticRegression(multi_class='multinomial',solver='newton-cg').fit(X_train,Y_train)
log_Y_pred = log.predict(X_test)
print("Logistc Accuracy: ",np.mean(log_Y_pred == np.array(Y_test)))
print(classification_report(Y_test,log_Y_pred))

##KNN
from sklearn.neighbors import KNeighborsClassifier
for neigh in range(1,5):
    knn = KNeighborsClassifier(n_neighbors=neigh).fit(X_train,Y_train)
    knn_Y_pred = knn.predict(X_test)
    print(neigh,"KNN Accuracy: ",np.mean(knn_Y_pred == np.array(Y_test)))
    print(classification_report(Y_test,knn_Y_pred))

## Linear SVC
from sklearn.svm import SVC
svcl = SVC(kernel='linear').fit(X_train,Y_train)
svcl_Y_pred = svcl.predict(X_test)
print("SVC Linear Accuracy: ",np.mean(svcl_Y_pred == np.array(Y_test)))
print(classification_report(Y_test,svcl_Y_pred))

## RBF SVC
from sklearn.svm import SVC
svcrbf = SVC(kernel='rbf', gamma=3).fit(X_train,Y_train)
svcrbf_Y_pred = svcrbf.predict(X_test)
print("SVC RBF Accuracy: ",np.mean(svcrbf_Y_pred == np.array(Y_test)))
print(classification_report(Y_test,svcrbf_Y_pred))

## Decision Tree
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier().fit(X_train,Y_train)
dt_Y_pred = dt.predict(X_test)
print("Decision Tree Accuracy: ",np.mean(dt_Y_pred == np.array(Y_test)))
print(classification_report(Y_test,dt_Y_pred))

## Random Forests
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier().fit(X_train,Y_train)
rf_Y_pred = rf.predict(X_test)
print("Random Forests Accuracy: ",np.mean(rf_Y_pred == np.array(Y_test)))
print(classification_report(Y_test,rf_Y_pred))

# Neural Network

In [ ]:
# food = 1
# service = 2
# ambiance = 3
# others = 4

def integerize(x):
    if x == 'food':
        return 1
    elif x == 'service':
        return 2
    elif x == 'ambiance':
        return 3
    else:
        return 4

In [ ]:
df4 = df3_tf.copy()

In [ ]:
df4.category = df3_tf.category.apply(integerize)

In [ ]:
train_nn, test_nn = train_test_split(df4,test_size = 0.25)
X_train_nn = train_nn.iloc[:,2:].reset_index(drop=True)
Y_train_nn = train_nn.iloc[:,1].reset_index(drop=True)
X_test_nn = test_nn.iloc[:,2:].reset_index(drop=True)
Y_test_nn = test_nn.iloc[:,1].reset_index(drop=True)

In [ ]:
from sknn.mlp import Classifier, Layer

nn = Classifier(
    layers=[
        Layer("Tanh", units=200),
        Layer("Softmax")],
    learning_rate=0.001,
    n_iter=50,
    loss_type='mcc')
nn.fit(df4.iloc[:,2:], df4.iloc[:,1])

In [ ]:
y_pred = nn.predict(df4.iloc[:,2:])

In [ ]:
print("NN: ",np.mean(y_pred == np.array(df4.iloc[:,1])))
print(classification_report(df4.iloc[:,1],y_pred))

# Vectorizing New Text

In [42]:
new_text = df2.sentence

In [57]:
vocab_list = tfvectorizer.get_feature_names()

In [58]:
vocab_dict = dict.fromkeys(vocab_list)

In [59]:
tfvectorizer2 = TfidfVectorizer(ngram_range=(1,3),min_df = 3,tokenizer=tokenizeText,sublinear_tf=True,vocabulary=vocab_dict)

In [60]:
len(vocab_list)

208

In [68]:
check = tfvectorizer.transform(new_text[0:100])

In [69]:
check

<100x208 sparse matrix of type '<class 'numpy.float64'>'
	with 286 stored elements in Compressed Sparse Row format>